In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import time

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [2]:
url = "https://fbref.com/pt/equipes/e0652b02/Juventus-Estatisticas"

In [3]:
site = requests.get(url)

In [4]:
site

<Response [200]>

In [5]:
bs = BeautifulSoup(site.content, "html.parser")

In [6]:
league = bs.find("div", {"id": "meta"}).find("span", {"class":"header_end"}).get_text().replace(")", "").replace("(", "")
league

'Serie A'

In [7]:
table = bs.find("table", {"id":re.compile("stats_standard")})

In [8]:
club = table.find("span").get_text()
club = club[club.find(" "):club.find(":")].strip()
club

'Juventus'

In [10]:
columns = []
for column_name in table.thead.find_all("tr")[1]:
    result = column_name.get_text()
    if result == " ": #elimina espaços vazios
        continue
    columns.append(result)
columns

['Jogador',
 'Nação',
 'Pos.',
 'Idade',
 'MP',
 'Inícios',
 'Min.',
 '90s',
 'Gols',
 'Assis.',
 'G-PB',
 'PB',
 'PT',
 'CrtsA',
 'CrtV',
 'Gols',
 'Assis.',
 'G+A',
 'G-PB',
 'G+A-PB',
 'xG',
 'npxG',
 'xA',
 'npxG+xA',
 'xG',
 'xA',
 'xG+xA',
 'npxG',
 'npxG+xA',
 'Partidas']

# Buscando 1 Linha (1 Jogador)

In [18]:
row = table.tbody.find_all("tr")[1]

In [13]:
[data.get_text() for data in row.find_all("td")]

['nl NED',
 'ZG',
 '21',
 '31',
 '29',
 '2.674',
 '29.7',
 '3',
 '1',
 '3',
 '0',
 '0',
 '4',
 '1',
 '0,10',
 '0,03',
 '0,13',
 '0,10',
 '0,13',
 '1.6',
 '1.6',
 '0.5',
 '2,1',
 '0,05',
 '0,02',
 '0,07',
 '0,05',
 '0,07',
 'Partidas']

A única informação contida nas linhas que não está em td é o nome do jogador

In [14]:
row.th.get_text()

'Matthijs de Ligt'

In [19]:
all_data_row = [data.get_text() for data in row.find_all("td")]
all_data_row.insert(0, row.th.get_text())

In [20]:
all_data_row

['Matthijs de Ligt',
 'nl NED',
 'ZG',
 '21',
 '31',
 '29',
 '2.674',
 '29.7',
 '3',
 '1',
 '3',
 '0',
 '0',
 '4',
 '1',
 '0,10',
 '0,03',
 '0,13',
 '0,10',
 '0,13',
 '1.6',
 '1.6',
 '0.5',
 '2,1',
 '0,05',
 '0,02',
 '0,07',
 '0,05',
 '0,07',
 'Partidas']

## Definindo Funções de Busca Para Todas as Linhas das Tabelas

In [23]:
def get_table(url):
    
    site = requests.get(url)
    bs = BeautifulSoup(site.content, "html.parser")
    
    table = bs.find("table", {"id":re.compile("stats_standard")})
    
    league = bs.find("div", {"id": "meta"}).find("span", {"class":"header_end"}).get_text().replace(")", "").replace("(", "")
    club_name = table.find("span").get_text()
    club_name = club_name[club_name.find(" "):club_name.find(":")].strip()
    
    
    columns = []
    for column_name in table.thead.find_all("tr")[1]:
        result = column_name.get_text()
        if result == " ":
            continue
        columns.append(result)
    
    df = pd.DataFrame(columns=columns)
    for row in table.tbody.find_all("tr"):
        #para cada linha pega o nome
        name = row.find("th").get_text()
        #para cada linha pega todos os td
        result = [i.get_text() for i in row.find_all("td")]  #[1:]]
        
        result.insert(0,name)

        df.loc[len(df)] = result
        
    df["Liga"] = league
    df["Clube"] = club_name
    return df

In [24]:
def format_table(df):
    
    df = df.replace("", np.nan)
    df = df.dropna()

    for i in range(len(df.columns)):
        df.iloc[:,i] = df.iloc[:,i].apply(lambda x: str(x).replace(",","."))

    df["Min."] = df["Min."].apply(lambda x: str(x).replace(".",""))
    
    #ajustar pais
    df["Nação"] = df["Nação"].apply(lambda x: x[str(x).find(" "):].strip())
    
    df.iloc[:, 15:30] = np.nan
    df = df.dropna(axis=1)
    

    l1 = ["ZG", "LE", "LD", "CB"]
    d1 = dict.fromkeys(l1, 'DEF')

    l2 = ["LT", "MC", "ME", "MD", "GM", "MA"]
    d2 = dict.fromkeys(l2, 'MEI')

    l3 = ["AT", "PE", "PD"]
    d3 = dict.fromkeys(l3, 'ATA')

    d4 = {"G":"GK"}

    d5 = dict(d1)
    d5.update(d2)
    d5.update(d3)
    d5.update(d4)
    
    df["Pos."] = df["Pos."].str.split(".", expand=True).rename(columns={0:"Pos1",1:"Pos2"}).iloc[:,0].map(d5)
    
    
    cols_order = ['Jogador', 'Liga','Clube', 'Nação', 'Pos.', 'Idade', 'MP', 'Inícios', 'Min.', '90s', 'Gols', 'Assis.', 'G-PB', 'PB', 'PT', 'CrtsA', 'CrtV']
    df = df[cols_order]
        
    
    
    int_cols = ["Idade","MP","Inícios","Min.", "Gols","Assis.","G-PB", "PB", "PT","CrtsA","CrtV"]
    float_cols = ["90s"]
    
    for col in int_cols:
        df[col] = df[col].astype(int)
        
    for col in float_cols:
        df[col] = df[col].astype(float)
    

    df["G_90"] = (df["Gols"] / df["90s"]).round(2)
    df["A_90"] = (df["Assis."] / df["90s"]).round(2)
    df["G+A_90"] = ((df["Gols"] + df["Assis."]) / df["90s"]).round(2)
    
    #garantir que nao tenha NA
    df = df.dropna()
    
    return df


In [21]:
with open('urls_dic2.json', 'r') as f:
    data = json.load(f)

In [29]:
data

{'Manchester City': 'https://fbref.com/pt/equipes/b8fd03ef/Manchester-City-Estatisticas',
 'Liverpool': 'https://fbref.com/pt/equipes/822bd0ba/Liverpool-Estatisticas',
 'Bayern Munich': 'https://fbref.com/pt/equipes/054efa67/Bayern-Munich-Estatisticas',
 'Paris S-G': 'https://fbref.com/pt/equipes/e2d8892c/Paris-Saint-Germain-Estatisticas',
 'Real Madrid': 'https://fbref.com/pt/equipes/53a2f082/Real-Madrid-Estatisticas',
 'Milan': 'https://fbref.com/pt/equipes/dc56fe14/Milan-Estatisticas',
 'Inter': 'https://fbref.com/pt/equipes/d609edc0/Internazionale-Estatisticas',
 'Napoli': 'https://fbref.com/pt/equipes/d48ad4ff/Napoli-Estatisticas',
 'Dortmund': 'https://fbref.com/pt/equipes/add600ae/Dortmund-Estatisticas',
 'Chelsea': 'https://fbref.com/pt/equipes/cff3d9bb/Chelsea-Estatisticas',
 'Barcelona': 'https://fbref.com/pt/equipes/206d90db/Barcelona-Estatisticas',
 'Leverkusen': 'https://fbref.com/pt/equipes/c7a9f859/Bayer-Leverkusen-Estatisticas',
 'Tottenham': 'https://fbref.com/pt/equip

In [30]:
type(data)

dict

In [31]:
len(data)

98

In [25]:
df = get_table(data["Juventus"])

In [26]:
df = format_table(df)

In [27]:
df.head()

,Jogador,Liga,Clube,Nação,Pos.,Idade,MP,Inícios,Min.,90s,Gols,Assis.,G-PB,PB,PT,CrtsA,CrtV,G_90,A_90,G+A_90
0,Wojciech Szczęsny,Serie A,Juventus,POL,GK,31,33,33,2970,33.0,0,0,0,0,0,2,0,0.00,0.00,0.00
1,Matthijs de Ligt,Serie A,Juventus,NED,DEF,21,31,29,2674,29.7,3,1,3,0,0,4,1,0.10,0.03,0.13
2,Juan Cuadrado,Serie A,Juventus,COL,DEF,33,33,28,2487,27.6,4,3,4,0,0,9,0,0.14,0.11,0.25
3,Adrien Rabiot,Serie A,Juventus,FRA,MEI,26,32,28,2285,25.4,0,2,0,0,0,5,0,0.00,0.08,0.08
4,Álvaro Morata,Serie A,Juventus,ESP,ATA,28,35,26,2302,25.6,9,4,8,1,1,5,0,0.35,0.16,0.51


In [29]:
len(data)

98

## Executando Funções

In [6]:
start_time = time.time()

list_of_dfs = []
i = 0
for name, url in data.items():
    time.sleep(10)
    df = get_table(url)
    df = format_table(df)
    

    list_of_dfs.append(df)
    print(f"{name} --------- OK------------- {i} \ {len(data)} ")

    i+=1
final_df = pd.concat(list_of_dfs, ignore_index=True)
print(f"Tempo de execução: {round(time.time() - start_time, 2)} segundos")

Manchester City --------- OK------------- 0 \ 98 
Liverpool --------- OK------------- 1 \ 98 
Bayern Munich --------- OK------------- 2 \ 98 
Paris S-G --------- OK------------- 3 \ 98 
Real Madrid --------- OK------------- 4 \ 98 
Milan --------- OK------------- 5 \ 98 
Inter --------- OK------------- 6 \ 98 
Napoli --------- OK------------- 7 \ 98 
Dortmund --------- OK------------- 8 \ 98 
Chelsea --------- OK------------- 9 \ 98 
Barcelona --------- OK------------- 10 \ 98 
Leverkusen --------- OK------------- 11 \ 98 
Tottenham --------- OK------------- 12 \ 98 
Marseille --------- OK------------- 13 \ 98 
Atlético Madrid --------- OK------------- 14 \ 98 
Sevilla --------- OK------------- 15 \ 98 
Juventus --------- OK------------- 16 \ 98 
Monaco --------- OK------------- 17 \ 98 
Arsenal --------- OK------------- 18 \ 98 
Rennes --------- OK------------- 19 \ 98 
Nice --------- OK------------- 20 \ 98 
RB Leipzig --------- OK------------- 21 \ 98 
Betis --------- OK------------

In [9]:
#Salvando para csv
#final_df.to_csv(".data/df_data_players.csv", index=False)

#Lendo csv
#final_df = pd.read_csv("./data/df_data_players.csv")

In [37]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Jogador  2858 non-null   object 
 1   Liga     2858 non-null   object 
 2   Clube    2858 non-null   object 
 3   Nação    2858 non-null   object 
 4   Pos.     2858 non-null   object 
 5   Idade    2858 non-null   int64  
 6   MP       2858 non-null   int64  
 7   Inícios  2858 non-null   int64  
 8   Min.     2858 non-null   int64  
 9   90s      2858 non-null   float64
 10  Gols     2858 non-null   int64  
 11  Assis.   2858 non-null   int64  
 12  G-PB     2858 non-null   int64  
 13  PB       2858 non-null   int64  
 14  PT       2858 non-null   int64  
 15  CrtsA    2858 non-null   int64  
 16  CrtV     2858 non-null   int64  
 17  G_90     2858 non-null   float64
 18  A_90     2858 non-null   float64
 19  G+A_90   2858 non-null   float64
dtypes: float64(4), int64(11), object(5)
memory usage: 44

In [122]:
final_df.head()

,Jogador,Liga,Clube,Nação,Pos.,Idade,MP,Inícios,Min.,90s,Gols,Assis.,G-PB,PB,PT,CrtsA,CrtV,G_90,A_90,G+A_90
0,Ederson,Premier League,Manchester City,BRA,GK,27,37,37,3330,37.0,0,0,0,0,0,3,0,0.00,0.00,0.00
1,João Cancelo,Premier League,Manchester City,POR,DEF,27,36,36,3227,35.9,1,7,1,0,0,7,0,0.03,0.19,0.22
2,Rodri,Premier League,Manchester City,ESP,MEI,25,33,33,2884,32.0,7,2,7,0,0,5,0,0.22,0.06,0.28
3,Bernardo Silva,Premier League,Manchester City,POR,MEI,26,35,33,2857,31.7,8,4,8,0,0,6,0,0.25,0.13,0.38
4,Aymeric Laporte,Premier League,Manchester City,ESP,DEF,27,33,33,2828,31.4,4,0,4,0,0,5,1,0.13,0.00,0.13
